More information on this link https://developer.nvidia.com/blog/how-overlap-data-transfers-cuda-cc/

In [ ]:
%%writefile vector_addition_stream.cu
#include <stdio.h>
#include <stdlib.h>
#include <type_traits>

#define TYPE int
#define N 51200

#define nStream 4

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void device_add(T *a, T *b, T *c, int n)
{
    int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        c[id] = a[id] + b[id];
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
void print_output(T *a, T *b, T *c)
{
    for (int i = 0; i < N; ++i)
    {
        printf("\n %d + %d  = %d", a[i], b[i], c[i]);
    }
}

int main()
{
    TYPE *a, *b, *c;
    TYPE *d_a, *d_b, *d_c;
    int threads_per_block = 0, no_of_blocks = 0;

    int size = N * sizeof(TYPE);

    // Allocate space for host copies of a, b, c and setup input values
    cudaHostAlloc(&a, size, cudaHostAllocDefault);
    cudaHostAlloc(&b, size, cudaHostAllocDefault);
    cudaHostAlloc(&c, size, cudaHostAllocDefault);

    for (int i = 0; i < N; ++i)
    {
        a[i] = i;
        b[i] = i;
    }

    // Allocate space for device copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // Create n streams
    cudaStream_t *streams = new cudaStream_t[nStream];
    for (int i = 0; i < nStream; ++i)
    {
        cudaStreamCreate(&streams[i]);
    }

    // Divide the work between n streams
    int partSize = N / nStream;
    int parts[nStream * 2];
    for (int i = 0; i < nStream; ++i)
    {
        parts[i * 2] = i * partSize; // Initial index
        parts[i * 2 + 1] = (i == nStream - 1) ? (N - (nStream - 1) * partSize) : partSize; // Element number of this part
    }

    // Copy data from host to device
    for (int i = 0; i < nStream; ++i)
    {
        cudaMemcpyAsync(d_a + parts[i * 2], a + parts[i * 2], parts[i * 2 + 1] * sizeof(TYPE), cudaMemcpyHostToDevice, streams[i]);
        cudaMemcpyAsync(d_b + parts[i * 2], b + parts[i * 2], parts[i * 2 + 1] * sizeof(TYPE), cudaMemcpyHostToDevice, streams[i]);
    }

    // Execute kernel
    for (int i = 0; i < nStream; ++i)
    {
        threads_per_block = 128;
        no_of_blocks = (parts[i * 2 + 1] + threads_per_block - 1) / threads_per_block;
        device_add<<<no_of_blocks, threads_per_block, 0, streams[i]>>>(
            d_a + parts[i * 2], d_b + parts[i * 2], d_c + parts[i * 2], parts[i * 2 + 1]);
    }

    // Copy result back to host
    for (int i = 0; i < nStream; ++i)
    {
        cudaMemcpyAsync(c + parts[i * 2], d_c + parts[i * 2], parts[i * 2 + 1] * sizeof(TYPE), cudaMemcpyDeviceToHost, streams[i]);
    }

    // Another way to achieve the overlapping
    //  for (int i = 0; i < nStream; ++i)
    //  {
    //      cudaMemcpyAsync(d_a + parts[i * 2], a + parts[i * 2], parts[i * 2 + 1] * sizeof(TYPE), cudaMemcpyHostToDevice, streams[i]);
    //      cudaMemcpyAsync(d_b + parts[i * 2], b + parts[i * 2], parts[i * 2 + 1] * sizeof(TYPE), cudaMemcpyHostToDevice, streams[i]);
    //      threads_per_block = 128;
    //      no_of_blocks = (parts[i * 2 + 1] + threads_per_block - 1) / threads_per_block;
    //
    //      device_add<<<no_of_blocks, threads_per_block, 0, streams[i]>>>(
    //          d_a + parts[i * 2], d_b + parts[i * 2], d_c + parts[i * 2], parts[i * 2 + 1]);
    //      cudaMemcpyAsync(c + parts[i * 2], d_c + parts[i * 2], parts[i * 2 + 1] * sizeof(TYPE), cudaMemcpyDeviceToHost, streams[i]);
    //  }

    cudaDeviceSynchronize();

    // print_output(a, b, c);

    // Clean up
    for (int i = 0; i < nStream; ++i)
    {
        cudaStreamDestroy(streams[i]);
    }
    delete[] streams;
    cudaFreeHost(a);
    cudaFreeHost(b);
    cudaFreeHost(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

In [13]:
!nvcc -o vector_addition_stream vector_addition_stream.cu

In [3]:
!./vector_addition_stream

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt update
!apt install ./drive/MyDrive/Nsight/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

In [ ]:
!nsys profile -o report_vector_addition_stream ./vector_addition_stream